# <code>geo_bezier_3d</code>
------------------------------------
## Nozzle

Para construção do Nozzle serão usados revolves e superfícies de Bézier. 

Todo o processo será comentado e explicado. Caso surjam dúvidas o usuário pode conferir **Docstring** e **How To** na [documentação](https://geo-bezier-3d.readthedocs.io/). 

<img src="images/nozzle.png" width="450"/>

### 1. Primeiros passos

Bem como na documentação (How To), inicialmente o usuário deverá setar os parâmetros do domínio, dar nome ao projeto, importar bibliotecas e arquivos e setar as informações de refinamento:

In [ ]:
#lendo de outra forma: 'eixo_x'=[comprimento lx, número de nós nx, is_periodic]

domain_info={'x':[10,91,False], 
             'y':[10,91,False],
             'z':[10,91,False]}

In [ ]:
for axis,(length,nodes,is_periodic) in domain_info.items():
    if is_periodic==True:
        domain_info["%s"%(axis)]+=[length/(nodes)]
    else:
        domain_info["%s"%(axis)]+=[length/(nodes-1)]

In [ ]:
name = str(input('Give your project a name: ')) #nome do projeto

In [ ]:
open(f'inputs.py', 'w').close()
with open(f'inputs.py', 'a') as the_file:
    the_file.write(f'domain_info={domain_info}\n')
    the_file.write('name='),the_file.write(f'"'), the_file.write(f'{name}'), the_file.write(f'"')

In [ ]:
#importação de bibliotecas e arquivos auxiliares do código

from infos import *
from inputs import *
from creating_solid import *
import inputs as i
import infos as s
import creating_solid as c

In [ ]:
gen_raf_information(nraf=2) #valor de nraf, o fator que refinará a malha em cada direção.

raf=False #para construir as epsis refinadas, apenas alterar essa variável para True

### 2. Criação dos limites

Setados os parâmetros iniciais, o usuário construirá de fato o sólido.

Obviamente, trata-se de um objeto bem complexo. Esse exemplo foi construído em um tempo não suficientemente longo para fazer com que a matriz $\epsilon$ final seja **exatamente** igual ao objeto apresentado. Algumas simplificações foram feitas.

Inicialmente, será construído um revolve:

In [ ]:
%matplotlib qt

In [ ]:
#limpando possível sujeira, boa prática

c.eq_storage={}
c.list_storage={}

#setando os dicionários essenciais para o revolve:

c.superior_revolve_info={'0':[[[0.000,4.125],[0.750,4.125]]], #cada linha é segmento de reta distinto, com 2 pontos cada
                         '1':[[[0.750,4.125],[5.048,1.935]]],
                         '2':[[[5.048,1.935],[7.122,1.750]]],
                         '3':[[[7.122,1.727],[7.238,1.727]]],
                         '4':[[[7.238,1.727],[7.858,1.607]]],
                         '5':[[[7.858,1.607],[8.122,1.453]]], 
                         '6':[[[8.122,1.453],[9.774,0.958]]]
                         }

c.inferior_revolve_info={'0':[[[0.000,3.000],[2.292,3.000]]],
                         '1':[[[2.292,3.000],[2.600,2.902]]], 
                         '2':[[[2.600,2.902],[4.534,1.916]]],
                         '3':[[[4.534,1.916],[5.633,1.501],[6.732,1.286]]], # exceto por essa linha, que tem 3 pontos (é uma curva)
                         '4':[[[6.732,1.286],[9.774,0.858]]]
                        }

#uma vez setados os dicionários, setando de fato a feature:

gen_revolve_profile(identif='0',
                    name='revolve_cru',
                    direction='z',
                    center_1=5.0,
                    center_2=5.0,
                    init_height=0.0,
                    rev_raf_path=raf,
                    deflection=True)

#com o deflection ligado a deflexão é realizada, fazendo com que a curva de Bézier passe pelo ponto intermediário

Por boa prática também, o usuário deve conferir se para cada linha vermelha pontilhada há uma linha cinza contínua **dentro** do perfil que sofrerá revolução. Em alguns casos a função pode falhar, apresenta instabilidade em alguns casos.

Uma vez criado o revolve, deve-se criar as superfícies de Bézier que fatiarão a parte final do Nozzle, criando o efeito de "garra".

O processo consiste em determinar apenas os pontos de duas superfície e rotacioná-las, bem como em um **Pattern**:

In [ ]:
#para deixar que o próprio loop identifique as superfícies para o usuário, setar identif=1 e a cada superfície somar 1

identif=1

#para cada um dos loops criar duas superfícies:

for angle,nome in [0,'superficie_normal'], [60,'superficie_rot_60'], [120,'superficie_rot_120']:
    
    #uma superifice padrão

    set_point_matrix(num_u_points=2,num_v_points=2) 

    point_storage['P00']=[ 0.660+5, 1.144+5,8.912]
    point_storage['P01']=[-0.660+5,-1.144+5,8.912]
    point_storage['P10']=[-0.000+5, 0.924+5,9.774]
    point_storage['P11']=[-0.800+5,-0.462+5,9.774]

    create_point_matrix()
    
    rotate(plane='xy',origin=[5,5],angle=angle)
    
    gen_bezier_surface(identif=str(identif),name=nome)
    
    identif+=1
    
    #outra superifice padrão

    set_point_matrix(num_u_points=2,num_v_points=2) #from set_point_matrix to gen_bezier: creating a surface

    point_storage['P00']=[ 0.660+5, 1.144+5,8.912]
    point_storage['P01']=[-0.660+5,-1.144+5,8.912]
    point_storage['P10']=[ 0.800+5, 0.462+5,9.774]
    point_storage['P11']=[-0.000+5,-0.924+5,9.774]

    create_point_matrix()
    
    rotate(plane='xy',origin=[5,5],angle=angle)
    
    gen_bezier_surface(identif=str(identif),name=nome)
    
    identif+=1

Se no início do loop <code>identif</code> valia 1 e o loop é feito 3 vezes somando 2 unidades a cada loop, <code>identif</code> final é 6.

Para visualizar tais superfícies criadas:

In [ ]:
surface_plot(init_identif='1', final_identif='7', engine='matplotlib', points=False, alpha=1) #try 'mayavi' in 1st argument!

Uma vez criadas todas as features que construirão o objeto desejado, é hora de validar tais limites na matriz $\epsilon$:

In [ ]:
#validando o revolve na epsi

gen_epsi_revolve(identif='0')

#validando as superfícies de Bézier criadas como saída, ou seja, a partir dela não existe mais sólido.
#o referencial para tal condição é a origem de plane, logo, a origem do plano xy (onde z=0).

for identif in range(1,7,1): #começa em 1, vai até 7 sem incluir 7, com passo de 1
    
    gen_epsi_bezier_surface(surface_type='entry+exit and/or exit',plane='xy',identif=str(identif),bez_raf_path=raf)


Por fim, criar os arquivos de saída para visualização do projeto no ParaView:

In [ ]:
gen_output(names=name,out_raf_path=raf)

O usuário pode perceber que para uma superfície muito fina (como o final do revolve) a representação da $\epsilon$ no ParaView não fica das melhores. Para resolver o problema, deve-se ou aumenta a espessura ou aumentar o número de nós da malha cartesiada tridimensional.